## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'JobSatisfaction', 'NumCompaniesWorked', 'OverTime', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1470 non-null   int64 
 1   DistanceFromHome         1470 non-null   int64 
 2   Education                1470 non-null   int64 
 3   JobSatisfaction          1470 non-null   int64 
 4   NumCompaniesWorked       1470 non-null   int64 
 5   OverTime                 1470 non-null   object
 6   TotalWorkingYears        1470 non-null   int64 
 7   WorkLifeBalance          1470 non-null   int64 
 8   YearsAtCompany           1470 non-null   int64 
 9   YearsSinceLastPromotion  1470 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 115.0+ KB


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'No': 0, 'Yes': 1})
X_test['OverTime'] = X_test['OverTime'].map({'No': 0, 'Yes': 1})


In [7]:
X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1176 entries, 1097 to 1126
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      1176 non-null   int64
 1   DistanceFromHome         1176 non-null   int64
 2   Education                1176 non-null   int64
 3   JobSatisfaction          1176 non-null   int64
 4   NumCompaniesWorked       1176 non-null   int64
 5   OverTime                 1176 non-null   int64
 6   TotalWorkingYears        1176 non-null   int64
 7   WorkLifeBalance          1176 non-null   int64
 8   YearsAtCompany           1176 non-null   int64
 9   YearsSinceLastPromotion  1176 non-null   int64
dtypes: int64(10)
memory usage: 101.1 KB
<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 1041 to 651
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age               

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])
department_train_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [10]:
# Create a DataFrame with the encoded columns
department_encoded_columns = department_encoder.get_feature_names_out(['Department'])
department_train_encoded_df = pd.DataFrame(department_train_encoded, columns=department_encoded_columns)
department_test_encoded_df = pd.DataFrame(department_test_encoded, columns=department_encoded_columns)
department_train_encoded_df

,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
1171,0.0,1.0,0.0
1172,0.0,1.0,0.0
1173,0.0,1.0,0.0
1174,0.0,1.0,0.0


In [11]:
# Remove the original Department column from y_train and y_test
y_train = y_train.drop(columns=['Department'])
y_test = y_test.drop(columns=['Department'])
# Add the new encoded columns to y_train and y_test
y_train = pd.concat([y_train, department_train_encoded_df], axis=1)
y_test = pd.concat([y_test, department_test_encoded_df], axis=1)

In [12]:
y_train.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 1097 to 1168
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Attrition                          1176 non-null   object 
 1   Department_Human Resources         1176 non-null   float64
 2   Department_Research & Development  1176 non-null   float64
 3   Department_Sales                   1176 non-null   float64
dtypes: float64(3), object(1)
memory usage: 55.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 534 entries, 1041 to 293
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Attrition                          294 non-null    object 
 1   Department_Human Resources         294 non-null    float64
 2   Department_Research & Development  294 non-null    float64
 3   Department_Sales                

In [13]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])
attrition_train_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [14]:
# Create a DataFrame with the encoded columns

attrition_encoded_columns = attrition_encoder.get_feature_names_out(['Attrition'])
attrition_train_encoded_df = pd.DataFrame(attrition_train_encoded, columns=attrition_encoded_columns)
attrition_test_encoded_df = pd.DataFrame(attrition_test_encoded, columns=attrition_encoded_columns)

In [15]:
# Remove the original Attrition column from y_train and y_test
# Add the new encoded columns to y_train and y_test

y_train = y_train.drop(columns=['Attrition'])
y_test = y_test.drop(columns=['Attrition'])
y_train = pd.concat([y_train, attrition_train_encoded_df], axis=1)
y_test = pd.concat([y_test, attrition_test_encoded_df], axis=1)

In [16]:
y_train.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1464 entries, 1097 to 1419
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Department_Human Resources         1176 non-null   float64
 1   Department_Research & Development  1176 non-null   float64
 2   Department_Sales                   1176 non-null   float64
 3   Attrition_No                       1420 non-null   float64
 4   Attrition_Yes                      1420 non-null   float64
 5   Attrition_nan                      1420 non-null   float64
dtypes: float64(6)
memory usage: 80.1 KB
<class 'pandas.core.frame.DataFrame'>
Index: 719 entries, 1041 to 533
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Department_Human Resources         294 non-null    float64
 1   Department_Research & Development  294 non-n

## Part 2: Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data.
n_features = X_train.shape[1]
print(n_features)

# Create the input layer
input_layer = layers.Input(shape=(n_features,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

10


In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
shared_layer3 = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(1, activation='sigmoid')(shared_layer3)

In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
shared_layer4 = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid')(shared_layer4)

In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
               loss={'department_output' : 'categorical_crossentropy', 'attrition_output' : 'binary_crossentropy'},
                 metrics={'department_output' : 'accuracy', 'attrition_output' : 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         17 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         17 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,874 (15.13 KB)

 Trainable params: 3,874 (15.13 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


ValueError: y_true and y_pred have different structures.
y_true: *
y_pred: ['*', '*']


In [ ]:
# Evaluate the model with the testing data


In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 